In [3]:
%%writefile constant.cu

#include <stdio.h>

// Declare a constant memory on the device for USD conversion rates

__constant__ float conversionRates[5];



// Define an enum for currency codes

enum CurrencyCode {INR=0, EUR=1, GBP=2, JPY=3, AUD=4};



// Kernel function to calculate the USD equivalents for multiple transactions

__global__ void converttoUSD(float *output, const float *amounts, const int *currencyIndices, int numTransactions) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < numTransactions && currencyIndices[idx] >= INR && currencyIndices[idx] <= AUD)
    {
        //Perform the conversion using the constant conversion rates

        output[idx] = amounts[idx] / conversionRates[currencyIndices[idx]];
    }
}

int main(){
   // Host array with conversion rates tp USD for fiver currencyIndices
   float h_conversionRates[5] = {82.0f, 0.95f, 0.78f, 130.0f, 1.5f};  //Example rates for INR, EUR, GBP, JPY, AUD

   // copy the rates to constant memory on device
   cudaMemcpyToSymbol(conversionRates, h_conversionRates, 5*sizeof(float));

   // Define 10 transaction amounts and their currency codes using the enum

   float h_amounts[10] = {50000.0f, 1000.0f, 2000.0f, 75000.0f, 1500.0f, 30000.0f, 200.0f, 10000.0f, 850.0f, 1200.0f};

   CurrencyCode h_currencyCodes[10] = {INR, EUR, GBP, INR, AUD, JPY, EUR, GBP, JPY, AUD};

   // convert the currency codes to integer currencyIndices

   int h_currencyIndices[10];

   for(int i=0; i<10; i++){
      h_currencyIndices[i] = (int)(h_currencyCodes[i]);
   }

   // Device arrays for transaction amounts, currency indices, and results

   float *d_amounts, *d_output;
   int *d_currencyIndices;
   float h_output[10];


   cudaMalloc(&d_amounts, 10*sizeof(float));
   cudaMalloc(&d_currencyIndices, 10*sizeof(int));
   cudaMalloc(&d_output, 10*sizeof(float));


   // copy transaction data to device

   cudaMemcpy(d_amounts, h_amounts, 10*sizeof(float), cudaMemcpyHostToDevice);
   cudaMemcpy(d_currencyIndices, h_currencyIndices, 10*sizeof(int), cudaMemcpyHostToDevice);

   // Launch the kernel with enough threads to handle 10 transactions

   int threadsPerBlock = 10;
   int blocksPerGrid = (10+threadsPerBlock-1)/threadsPerBlock;

   converttoUSD<<<blocksPerGrid, threadsPerBlock>>>(d_output, d_amounts, d_currencyIndices, 10);

   // copy results back to host

   cudaMemcpy(h_output, d_output, 10*sizeof(float), cudaMemcpyDeviceToHost);

   //Array of currency names

   const char* currencyNames[] = {"INR", "EUR", "GBP", "JPY", "AUD"};

   //Display the results with currency names

   for(int i=0; i<10; i++)
   {
      printf("Transaction %2d (%s %8.2f): Equivalent USD amount = %.2f\n", i+1, currencyNames[h_currencyIndices[i]], h_amounts[i], h_output[i]);
   }

   //clean up
   cudaFree(d_amounts);
   cudaFree(d_currencyIndices);
   cudaFree(d_output);

   return 0;
}

Overwriting constant.cu


In [4]:
!nvcc -o exe constant.cu
!./exe

Transaction  1 (INR 50000.00): Equivalent USD amount = 609.76
Transaction  2 (EUR  1000.00): Equivalent USD amount = 1052.63
Transaction  3 (GBP  2000.00): Equivalent USD amount = 2564.10
Transaction  4 (INR 75000.00): Equivalent USD amount = 914.63
Transaction  5 (AUD  1500.00): Equivalent USD amount = 1000.00
Transaction  6 (JPY 30000.00): Equivalent USD amount = 230.77
Transaction  7 (EUR   200.00): Equivalent USD amount = 210.53
Transaction  8 (GBP 10000.00): Equivalent USD amount = 12820.51
Transaction  9 (JPY   850.00): Equivalent USD amount = 6.54
Transaction 10 (AUD  1200.00): Equivalent USD amount = 800.00
